# Artificial Neural Network

We remind that if the user is using Google Colaboratory, he must change the execution type to GPU, see the README.

In [1]:
#Libraries for data processing
import numpy as np 
import pandas as pd

#Libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(color_codes = True)
sns.set(font_scale=1.5) #fixing font size

#Libraries for artificial neural network
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Normalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import Callback

2023-02-23 22:25:23.513974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 22:25:23.699537: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-23 22:25:23.705011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 22:25:23.705059: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

##### We import our training data (all nuclei except Z=10,38,54,68,82) and create rescaled inputs based on the dataframe columns for the ANN

In [2]:
train_data = pd.read_csv("3_rescaled_data/train_rescaled_data.csv", sep=";")

In [3]:
#First inputs
target = train_data["rescaled_ame_BE"]
n_input = train_data["rescaled_N"]
z_input = train_data["rescaled_Z"]

#Liquid drop inputs
surf_input = train_data["rescaled_Surf"]
asym_input = train_data["rescaled_Asym"]
coul_input = train_data["rescaled_Coul"]
pair_input = train_data["rescaled_Pair"]

#Other inputs that may help
z_parity_input = train_data["rescaled_Z_parity"]
n_parity_input = train_data["rescaled_N_parity"]
z_distance_input = train_data["rescaled_Z_distance"]  
n_distance_input = train_data["rescaled_N_distance"]
S1p_input = train_data["rescaled_ame_S1p"]
S1n_input = train_data["rescaled_ame_S1n"]
S2p_input = train_data["rescaled_ame_S2p"]
S2n_input = train_data["rescaled_ame_S2n"]

##### We create a function that will create the ANN. It takes 3 parameters: the number of inputs, the number of layers and the number of neurons per layer desired.

In [4]:
def create_model(num_inputs, num_layers, num_neurons):
    inputs = [keras.layers.Input(shape=(1,)) for i in range(num_inputs)]
    merged = keras.layers.Concatenate()(inputs)

    dense = merged
    for i in range(num_layers):
        dense = Dense(num_neurons, activation="relu")(dense)
    
    output = Dense(1, activation="relu")(dense)
    model = keras.models.Model(inputs, output)
    return model

##### We use this function to create an ANN with 12 inputs, 14 hidden layers with 100 neurons each.

In [5]:
model4 = create_model(12,14,100) 

2023-02-23 22:25:25.822863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 22:25:25.822929: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 22:25:25.822950: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (AlexianPC): /proc/driver/nvidia/version does not exist
2023-02-23 22:25:25.823222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model4.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error")

##### We create a class that will allow us to stop the training of the model when a specific loss is reach. Otherwise, the training will continue until the last epoch.

In [7]:
class EarlyStoppingByLossValue(Callback):
    def __init__(self, value=0.00000009):
        self.value = value

    def on_epoch_end(self, epoch, logs={}):
        current_loss = logs.get("loss")
        if current_loss < self.value:
            self.model.stop_training = True
            print("Early stopping by loss value at epoch", epoch)

early_stopping = EarlyStoppingByLossValue()

##### We train the model and plot the evolution of the loss among epochs.

Sometimes when compiling the next code bloc, the loss will be constant. It shouldn't be the case and thus the user has to stop the execution and compile again. It may take some retries before the loss decreases as intended.

In [8]:
history4=model4.fit(x=([n_input, z_input, surf_input, coul_input, asym_input,
                        pair_input, S1n_input, S1p_input, S2n_input, S2p_input, 
                        z_distance_input, n_distance_input]),
                        y=target, epochs=6000, shuffle=True,
                        verbose=2, callbacks=[early_stopping] )

plt.figure(figsize =(20,13))
plt.yscale('log')
plt.legend('labels')

plt.plot(history4.history["loss"])
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.show()
#Be careful : loss is mean_squared_error, not RMSE

Epoch 1/6000
84/84 - 2s - loss: 0.0340 - 2s/epoch - 21ms/step
Epoch 2/6000
84/84 - 0s - loss: 1.3257e-04 - 457ms/epoch - 5ms/step
Epoch 3/6000
84/84 - 0s - loss: 8.5722e-05 - 458ms/epoch - 5ms/step
Epoch 4/6000
84/84 - 0s - loss: 8.7786e-05 - 464ms/epoch - 6ms/step
Epoch 5/6000
84/84 - 0s - loss: 5.5693e-05 - 497ms/epoch - 6ms/step
Epoch 6/6000
84/84 - 0s - loss: 4.7692e-05 - 462ms/epoch - 6ms/step
Epoch 7/6000
84/84 - 1s - loss: 3.7497e-05 - 556ms/epoch - 7ms/step
Epoch 8/6000
84/84 - 0s - loss: 2.4558e-05 - 472ms/epoch - 6ms/step
Epoch 9/6000
84/84 - 0s - loss: 1.8476e-05 - 438ms/epoch - 5ms/step
Epoch 10/6000
84/84 - 0s - loss: 1.1875e-05 - 472ms/epoch - 6ms/step
Epoch 11/6000
84/84 - 0s - loss: 2.1868e-05 - 478ms/epoch - 6ms/step
Epoch 12/6000
84/84 - 0s - loss: 2.1242e-05 - 494ms/epoch - 6ms/step
Epoch 13/6000
84/84 - 0s - loss: 9.0463e-06 - 473ms/epoch - 6ms/step
Epoch 14/6000
84/84 - 1s - loss: 2.0162e-05 - 514ms/epoch - 6ms/step
Epoch 15/6000
84/84 - 0s - loss: 2.4184e-05 - 484

KeyboardInterrupt: 

##### We rescale the target data.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

rescaled_data = pd.read_csv("3_rescaled_data/rescaled_data.csv", sep=";")

rescaled_target = scaler.fit_transform(pd.Series.to_numpy(rescaled_data["ame_BE"]).reshape(-1,1))

#### Here, we predict the binding energy for all the nuclei used for training, and we compute the difference between the prediction and experimental data.

In [ ]:
train_predictions = model4.predict(x=([n_input, z_input, surf_input, coul_input,
                                       asym_input, pair_input, S1n_input, S1p_input,  
                                       S2n_input, S2p_input, z_distance_input,
                                       n_distance_input]), verbose=0)


train_rescaled_predictions = [(i - scaler.min_)/scaler.scale_ for i in train_predictions]


train_data["BE_Predictions"] = np.double(train_rescaled_predictions)
train_data["Difference_BE_AME_ANN"] = train_data["ame_BE"] - train_data["BE_Predictions"]
train_data["Difference_BE_DZ_ANN"] = train_data["dz_BE"] - train_data["ame_BE"]

##### We will now do the same operation as above, but we will use the model to predict binding energies for the nuclei it has never trained on (Z=10,38,54,68,82) (test data)

In [ ]:
test_data = pd.read_csv("3_rescaled_data/test_rescaled_data.csv", sep=";")

test_target = test_data["rescaled_ame_BE"]
test_n_input = test_data["rescaled_N"]
test_z_input = test_data["rescaled_Z"]
test_coul_input = test_data["rescaled_Coul"]
test_surf_input = test_data["rescaled_Surf"]
test_asym_input = test_data["rescaled_Asym"]
test_pair_input = test_data["rescaled_Pair"]
test_z_parity_input = test_data["rescaled_Z_parity"]
test_n_parity_input = test_data["rescaled_N_parity"]
test_z_distance_input = test_data["rescaled_Z_distance"]
test_n_distance_input = test_data["rescaled_N_distance"]
test_S1p_input = test_data["rescaled_ame_S1p"]
test_S1n_input = test_data["rescaled_ame_S1n"]
test_S2p_input = test_data["rescaled_ame_S2p"]
test_S2n_input = test_data["rescaled_ame_S2n"]

In [ ]:
test_predictions = model4.predict(x=([test_n_input, test_z_input,
                                            test_surf_input, test_coul_input,
                                            test_asym_input, test_pair_input,  
                                            test_S1n_input, test_S1p_input,
                                            test_S2n_input, test_S2p_input,
                                            test_z_distance_input,
                                            test_n_distance_input]))

test_rescaled_predictions = [ (i - scaler.min_)/scaler.scale_ for i in test_predictions]

test_data["BE_Predictions"] = np.double(test_rescaled_predictions)
test_data["Difference_BE_AME_ANN"] = test_data["ame_BE"] - test_data["BE_Predictions"]


##### Here, we compute the values for Sn, Sp, S2n et S2p with the predicted binding energies for both datasets (training and test)

In [ ]:
train_data=train_data.sort_values(by=['A','N'], ascending=True)

In [ ]:
train_data=train_data.sort_values(by=['N','Z'], ascending=True)
train_data['Prediction_S2p'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(2)

train_data = train_data.sort_values(by=['A','N'], ascending=True)
train_data['Prediction_S2n'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(2)
train_data['Prediction_S1n'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(1)

train_data["Difference_S2n_AME_Predictions"] = train_data["ame_S2n"] - train_data["Prediction_S2n"]
train_data["Difference_S2p_AME_Predictions"] = train_data["ame_S2p"] - train_data["Prediction_S2p"]

train_data["Difference_S2n_DZ_Predictions"] = train_data["dz_S2n"] - train_data["Prediction_S2n"]
train_data["Difference_S2p_DZ_Predictions"] = train_data["dz_S2p"] - train_data["Prediction_S2p"]

In [ ]:
test_data.sort_values(by=['N','Z'], ascending=True)
test_data['Prediction_S2p'] = test_data['BE_Predictions'] - test_data['BE_Predictions'].shift(2)

test_data = test_data.sort_values(by=['A','N'], ascending=True)
test_data['Prediction_S2n'] = test_data['BE_Predictions'] - test_data['BE_Predictions'].shift(2)
test_data['Prediction_S1n'] = test_data['BE_Predictions'] - test_data['BE_Predictions'].shift(1)

test_data["Difference_S2n_AME_Predictions"] = test_data["ame_S2n"] - test_data["Prediction_S2n"]
test_data["Difference_S2p_AME_Predictions"] = test_data["ame_S2p"] - test_data["Prediction_S2p"]

test_data["Difference_S2n_DZ_Predictions"] = test_data["dz_S2n"] - test_data["Prediction_S2n"]
test_data["Difference_S2p_DZ_Predictions"] = test_data["dz_S2p"] - test_data["Prediction_S2p"]

#### We save the predictions into .csv and we will plot them on another notebook

In [ ]:
train_final_csv = train_data.to_csv("4_final_data/train_final_data.csv",sep=";")
test_final_csv = test_data.to_csv("4_final_data/test_final_data.csv",sep=";")